This jupyter notebook shows how we prepared training database via DFT calculations.

In [ ]:
#Step 1: Loading modules

#!/usr/bin/env python
import os
import shutil
import pickle
from os import system as shell
import numpy as np
from ase.db import connect
from ase.calculators.espresso import Espresso
from ase import Atoms, io, Atom
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import bcc100, surface
from ase.units import Rydberg, Bohr
from ase.io.trajectory import Trajectory
from ase.build.surface import *

In [ ]:
#Step 2: Loading lattice constant values

with open('MaterialDict.pkl', 'rb') as f:
    data = pickle.load(f, encoding='bytes')
data = {k.decode('utf8'): v for k, v in data.items()}

In [ ]:
# Step 3: Creating surface

a = data['Ag'][b'qLattConst-PBE'] # Lattice constant

print('qLattConst-PBE = ', a)

atoms = fcc111('Ag', size=(2,2,4), a=a) # Creating an FCC 111 Ag surface

for i in range(1):
    atoms[-i-1].symbol ='Cu' # Replacing one Ag atom with Cu atom to form Cu SAA on Ag

atoms.center(vacuum=7.5, axis=2) # Adding 7.5 A vacuum for both z-axis directions
atoms.set_pbc([1,1,1]) # The system is periodically

constraint = []
constraint.append(FixAtoms(mask=[1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])) # Fixed bottom two layers of atoms
atoms.set_constraint(constraint)

In [ ]:
# Step 4: Initializing magnetic moments

mag_moms = []

for atom in atoms:
  mag = 1
  if atom.symbol == "Mn":
    mag = 3
  if atom.symbol == "Fe":
    mag = 3
  if atom.symbol == "Ni":
    mag = 1
  if atom.symbol == "Co":
    mag = 2
  mag_moms.append(mag)

atoms.set_initial_magnetic_moments(mag_moms)

In [ ]:
# Step 5: Setting number of bands and pseudopotentials

symbols = atoms.get_chemical_symbols()

nel = {}

psp_path = '/espresso6.5/q-e/esp-psp' # Path to your QE folder

for s in symbols:
   p = os.popen('egrep -i \'z\ valence|z_valence\' '+psp_path+'/'+s+'.UPF | tr \'"\' \' \'','r')
   for y in p.readline().split():
     if y[0].isdigit() or y[0]=='.':
       nel[s] = int(round(float(y)))
       break
   p.close()

lis = [nel[s] for s in symbols]

tot_electrons = sum(lis) # Total number of valence electrons

nbands_ = int(tot_electrons/2 + .2 * tot_electrons/2) * 2.0 + 10

nbands_ = round(nbands_) # make sure integer.

pseudopotentials = {s:s+'.UPF' for s in symbols}

In [ ]:
# Step 6: Setup calculator

input_data = {
  'control': {
    'pseudo_dir': psp_path,
    'prefix': 'calc',
    'outdir': '.',
    'tprnfor': True}, # need tefield and dipfield for dipole corrections.
  'system': {
    'ibrav': 0,
    'nbnd': nbands_,
    'ecutwfc': 500/Rydberg, # need to convert to qe units
    'ecutrho': 5000/Rydberg,
    'occupations': 'smearing',
    'smearing': 'fd',
    'degauss': .1/Rydberg,
    'nspin': 2,
    'input_dft': 'PBE'},
  'electrons': {
    'diagonalization': 'david',
    'conv_thr': 1e-6/Rydberg, # 1e-6 eV threshold
    'mixing_beta': 0.1,
    'electron_maxstep': 500}, # maximum number of self consistent cycles.
  'disk_io': 'low'} # automatically put into 'control'

calc = Espresso(pseudopotentials = pseudopotentials,
         kpts = (6, 6, 1),
         tprnfor=True,
         tstress=True,
         input_data=input_data)

In [ ]:
# Step 7: Performing DFT calculations

# Restart calculation if previous calculation results exists.
if os.path.exists('qn.traj') and os.path.getsize('qn.traj') != 0:
  atoms = io.read('qn.traj', index=-1)

atoms.set_calculator(calc)

calc.command = 'mpirun pw.x -in espresso.pwi > espresso.pwo'

io.write("InitialGeom.traj", atoms)

Traj = Trajectory('qn.traj', 'a', atoms)

dyn = QuasiNewton(atoms, trajectory=Traj,
         logfile='qn.log', restart='qn.pckl')

dyn.run(fmax=0.1)

io.write("RelaxedGeom.traj", atoms)

In [ ]:
# Step 8: Performing DFT calculations

# nscf cycle with high k-points
input_data['control'].update({'calculation':'nscf'})

tight_calc = Espresso(label = 'Tight',
           pseudopotentials = pseudopotentials,
           kpts = (12, 12, 1),
           input_data = input_data)

atoms.set_calculator(tight_calc)

tight_calc.command = 'mpirun pw.x -in Tight.pwi > Tight.pwo'

tight_calc.calculate(atoms)

In [ ]:
# Step 9: Calculating density of states (DOS).
shell('mpirun projwfc.x -in pdos.inp')

shell('mpirun pp.x -in wf.inp > wf.out')

shell('average.x -in avg.inp > avg.out')

shell('rm ./calc.save/wfc*.dat') # Delete wfc files to save disk space

In [ ]:
# Step 10: Converting calc.pdos_tot files to pickle files

# Loading modules
import os
import numpy as np
from ase import io
import subprocess as sp
import pickle
import multiprocessing

In [ ]:
# Grabbing Fermi levels of 6 x 6 x 1 systems

def grab_fermi(path):
  encoding = "utf-8"
  p1 = sp.Popen(["grep", "Fermi energy", 
          path + "/espresso.pwo"], stdout = sp.PIPE)
  p2 = sp.Popen(['tail', '-n 1'], stdin = p1.stdout, 
         stdout = sp.PIPE)
  out = p2.communicate()
  str_ = out[0].decode(encoding)
  return float(str_.split()[-2])

In [ ]:
# Grabbing Fermi levels of 12 x 12 x 1 systems

def grab_fermi_tight(path):
  encoding = "utf-8"
  p1 = sp.Popen(["grep", "Fermi energy", 
          path + "/Tight.pwo"], stdout = sp.PIPE)
  p2 = sp.Popen(['tail', '-n 1'], stdin = p1.stdout, 
         stdout = sp.PIPE)
  out = p2.communicate()
  str_ = out[0].decode(encoding)
  return float(str_.split()[-2])

In [ ]:
# Converting calc.pdos_tot files to pickle files

def dos(file):
  try:
      home = os.getcwd()
      os.chdir(home)
      os.chdir(file)
      dos = np.loadtxt('calc.pdos_tot')   
      if len(dos[0])>3:
        nspin = 2
        dos_total = [dos[:,1],dos[:,2]]
      else:
        nspin = 1
        dos_total = dos[:,1]
      efermi_tight = grab_fermi_tight('.')
      efermi = grab_fermi('.')
      dos_energies = dos[:,0] - efermi_tight
      npoints = len(dos_energies)
      
      atoms = io.read('espresso.pwo')
      channels = {'s':0, 'p':1, 'd':2, 'f':3}
      pdos = [{} for i in range(len(atoms))]
      p = os.popen('ls calc.pdos_atm*')
      proj = p.readlines()
      p.close()
      proj.sort()
      add_higher_channels = True
      for i,inp in enumerate(proj):
        inpfile = inp.strip()
        pdosinp = np.genfromtxt(inpfile)
        spl = inpfile.split('#')
        iatom = int(spl[1].split('(')[0]) - 1
        channel = spl[2].split('(')[1].rstrip(')').replace('_j',',j=')
        jpos = channel.find('j=')
        if jpos<0:
          ncomponents = (2*channels[channel[0]]+2) * nspin
        else:
          ncomponents = int(2.*float(channel[jpos+2:]))+2
        if not channel in pdos[iatom]:
          pdos[iatom][channel] = np.zeros((ncomponents, npoints), np.float)
          first = True
        else:
          first = False
        if add_higher_channels or first:
          for j in range(ncomponents):
            pdos[iatom][channel][j] += pdosinp[:, (j+1)]
      pickle.dump([dos_energies, dos_total, pdos], open('dos.pickle', 'wb'))
      os.chdir(home)
      return file, efermi_tight, efermi
  except:
      os.chdir(home)

In [ ]:
# Executing code

files = [file for file in os.listdir('.') if os.path.isdir(file)] # Read folder names

ans = multiprocessing.Pool().map(dos, files) # Convert the calc.pdos_tot file in each folder

for i in ans:
    if i != None:
        print(i[0], i[1], i[2]) # Print out the folder name and Fermi levels for each system

Through the above code, you will get three important files:

(1) qn.traj

    Trajectory file of relaxation path. It will be used to check if the system is distorted after relaxation.

(2) qn.log

    Log file. It will be used to check whether the system has converged to the relaxation status.

(3) dos.pickle

    Contains the density of states for each orbital.